In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

In [ ]:
df = pd.read_csv('/kaggle/input/heart-disease-uci/heart.csv')

In [ ]:
df.head()

In [ ]:
df['age'].value_counts()

***Exploratory data analysis***

In [ ]:
df = df[df['thal'] != 0]
df = df[df['age'] != 29]
df.head()

In [ ]:
df['thal'] = df['thal'].replace(1, 'fixed defect')
df['thal'] = df['thal'].replace(2, 'normal')
df['thal'] = df['thal'].replace(3, 'reversable defect')
df['cp'] = df['cp'].replace(0, 'asymptomatic')
df['cp'] = df['cp'].replace(1, 'atypical angina')
df['cp'] = df['cp'].replace(2, 'non-anginal pain')
df['cp'] = df['cp'].replace(3, 'typical angina')
df['restecg'] = df['restecg'].replace(0, 'ventricular hypertrophy')
df['restecg'] = df['restecg'].replace(1, 'normal')
df['restecg'] = df['restecg'].replace(2, 'ST-T wave abnormality')
df['slope'] = df['slope'].replace(0, 'downsloping')
df['slope'] = df['slope'].replace(1, 'flat')
df['slope'] = df['slope'].replace(2, 'upsloping')

In [ ]:
temp = pd.get_dummies(df[['cp', 'restecg', 'slope', 'thal']])
df = df.join(temp, how='left')
df = df.drop(columns = ['cp','restecg', 'slope', 'thal'], axis=1)
df.head()

In [ ]:
df = df.drop(columns = ['restecg_ventricular hypertrophy', 'slope_upsloping', 'thal_fixed defect', 'cp_typical angina'], axis=1)
df.head()

In [ ]:
sns.countplot(df.target)
df.target.value_counts()

In [ ]:
X = df.drop(['target'], axis=1)
Y = df['target']
train_features, test_features, train_labels, test_labels = train_test_split(X, Y, test_size=0.2, random_state=0)

In [ ]:
rf = RandomForestClassifier()
dt = DecisionTreeClassifier()
lr = LogisticRegression()
knn = KNeighborsClassifier()

In [ ]:
rf.fit(train_features, train_labels)
dt.fit(train_features, train_labels)
lr.fit(train_features, train_labels)
knn.fit(train_features, train_labels)

In [382]:
rf_pred_train = rf.predict(train_features)
dt_pred_train = dt.predict(train_features)
lr_pred_train = lr.predict(train_features)
knn_pred_train = knn.predict(train_features)
rf_pred_test = rf.predict(test_features)
dt_pred_test = dt.predict(test_features)
lr_pred_test = lr.predict(test_features)
knn_pred_test = knn.predict(test_features)

In [383]:
rf_prob = rf.predict_proba(test_features)[:,1]
dt_prob = dt.predict_proba(test_features)[:,1]
lr_prob = lr.predict_proba(test_features)[:,1]
knn_prob = knn.predict_proba(test_features)[:,1]

In [384]:
rf_prob

array([0.6, 0.4, 1. , 0. , 0.6, 0.2, 0. , 0.5, 0.9, 0.7, 0.7, 0.9, 0. ,
       0.1, 0.6, 0.7, 0.9, 1. , 0. , 0.2, 0.2, 1. , 0.5, 1. , 0. , 1. ,
       0.4, 0.1, 0. , 0.4, 0.8, 1. , 0.9, 1. , 0.3, 0.5, 0.1, 0.1, 1. ,
       0.3, 0.2, 0.2, 0.6, 0.6, 0.9, 0.7, 0.2, 0.7, 0.7, 1. , 0.7, 0.1,
       0.3, 0.9, 1. , 0.1, 0.6, 0.8, 0.9, 0.8])

In [388]:
print(classification_report(test_labels,rf_pred_test))
print('Random Forest baseline: ' + str(roc_auc_score(train_labels, rf_pred_train)))
print('Random Forest: ' + str(roc_auc_score(test_labels, rf_pred_test)))
print(classification_report(test_labels,dt_pred_test))
print('Decision Tree baseline: ' + str(roc_auc_score(train_labels, dt_pred_train)))
print('Decision Tree: ' + str(roc_auc_score(test_labels, dt_pred_test)))
print(classification_report(test_labels,lr_pred_test))
print('Logistic Regression baseline: ' + str(roc_auc_score(train_labels, lr_pred_train)))
print('Logistic Regression: ' + str(roc_auc_score(test_labels, lr_pred_test)))
print(classification_report(test_labels,knn_pred_test))
print('KNN baseline: ' + str(roc_auc_score(train_labels, knn_pred_train)))
print('KNN: ' + str(roc_auc_score(test_labels, knn_pred_test)))

              precision    recall  f1-score   support

           0       0.81      0.76      0.79        29
           1       0.79      0.84      0.81        31

    accuracy                           0.80        60
   macro avg       0.80      0.80      0.80        60
weighted avg       0.80      0.80      0.80        60

Random Forest baseline: 0.9848484848484849
Random Forest: 0.7986651835372637
              precision    recall  f1-score   support

           0       0.79      0.66      0.72        29
           1       0.72      0.84      0.78        31

    accuracy                           0.75        60
   macro avg       0.76      0.75      0.75        60
weighted avg       0.76      0.75      0.75        60

Decision Tree baseline: 0.9848484848484849
Decision Tree: 0.7469410456062292
              precision    recall  f1-score   support

           0       0.82      0.79      0.81        29
           1       0.81      0.84      0.83        31

    accuracy                